In [ ]:
# scanpy
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import anndata as ad
import scanpy.external as sce
import os
import re
import warnings
warnings.filterwarnings("ignore")
sc.logging.print_header()
sc.__version__
ad.__version__
ir.__version__
import tarfile
import warnings
from glob import glob
import anndata
import matplotlib.pyplot as plt
import muon as mu
import pandas as pd
import scanpy as sc
import scirpy as ir

In [ ]:
## 1. input data
adata = sc.read("/share/home/qinxiaohuan/proj/06.CART_SLE/01.h5ad/UCART_SLE_6Samples_QC.h5ad")
print('before filter gene', adata.var)


# filter matrix with drop 'ENSG/LINC'
adata.var["gene_symbols"] = adata.var.index
adata.var["ENSG"]=adata.var_names.str.startswith('ENSG')
adata.var["LINC"]=adata.var_names.str.startswith('LINC')
adata = adata[:,~adata.var["ENSG"] & ~adata.var["LINC"]]
print('after filter gene', adata.var) # 23384

# filter biased genes
sc.pp.filter_genes(adata, min_cells = 10)
print(adata.var) # 27811


# 2. normalize and select the highly_vaeiable_genes
sc.pp.normalize_total(adata,target_sum = 1e4)
sc.pp.log1p(adata)


# 3. calculate geneset score
# cellcycle
sc.tl.score_genes_cell_cycle(adata,s_genes=["MCM5","PCNA","TYMS","FEN1","MCM2","MCM4","RRM1","UNG","GINS2","MCM6","CDCA7","DTL","PRIM1","UHRF1","HELLS","RFC2","RPA2","NASP","RAD51AP1","GMNN","WDR76","SLBP","CCNE2","POLD3","MSH2","ATAD2","RAD51","RRM2","CDC45","CDC6","EXO1","TIPIN","DSCC1","BLM","CASP8AP2","USP1","CLSPN","POLA1","CHAF1B","BRIP1","E2F8"],g2m_genes=["HMGB2","CDK1","NUSAP1","UBE2C","BIRC5","TPX2","TOP2A","NDC80","CKS2","NUF2","CKS1B","MKI67","TMPO","CENPF","TACC3","SMC4","CCNB2","CKAP2L","CKAP2","AURKB","BUB1","KIF11","ANP32E","TUBB4B","GTSE1","KIF20B","HJURP","HJURP","CDCA3","CDC20","TTK","CDC25C","KIF2C","RANGAP1","NCAPD2","DLGAP5","CDCA2","CDCA8","ECT2","KIF23","HMMR","AURKA","PSRC1","ANLN","LBR","CKAP5","CENPE","CTCF","NEK2","G2E3","GAS2L3","CBX5","CENPA"])
# calculate heatshock gene score
human_heatshock_genes = ["BBS10","BBS12","TCP1","CCT2","CCT3","CCT4","CCT5","CCT6A","CCT6B","CCT7","CCT8","CLPB","HSPD1","HSPE1","MKKS","DNAJA1","DNAJA2","DNAJA3","DNAJA4","DNAJB1","DNAJB2","DNAJB3","DNAJB4","DNAJB5","DNAJB6","DNAJB7","DNAJB8","DNAJB9","DNAJB11","DNAJB12","DNAJB13","DNAJB14","DNAJC1","DNAJC2","DNAJC3","DNAJC4","DNAJC5","DNAJC5B","DNAJC5G","DNAJC6","DNAJC7","DNAJC8","DNAJC9","DNAJC10","DNAJC11","DNAJC12","DNAJC13","DNAJC14","DNAJC15","DNAJC16","DNAJC17","DNAJC18","DNAJC19","HSCB","DNAJC21","DNAJC22","SEC63","DNAJC24","DNAJC25","GAK","DNAJC27","DNAJC28","SACS","DNAJC30","HSPA1A","HSPA1B","HSPA1L","HSPA2","HSPA4","HSPA4L","HSPA5","HSPA6","HSPA7","HSPA8","HSPA9","HSPA12A","HSPA12B","HSPA13","HSPA14","HSPH1","HYOU1","HSP90AA1","HSP90AA3P","HSP90AB1","HSP90B1","TRAP1","HSPB1","HSPB2","HSPB3","CRYAA","CRYAB","HSPB6","HSPB7","HSPB8","HSPB9","ODF1","IFT25"]
sc.tl.score_genes(adata, gene_list = human_heatshock_genes, score_name = "HSP_score")
# calculate globin gene score
human_globin_genes = ["HBB","HBD","HBA1","HBA2","HBG1","HBG2","HBQ1","HBM","CA1","ALAS2","AHSP","BLVRB"]
sc.tl.score_genes(adata, gene_list = human_globin_genes, score_name = "Globin_score")
# MALAT1
human_MALAT1 = ["MALAT1"]
sc.tl.score_genes(adata, gene_list = human_MALAT1, score_name = "MALAT1_score")


# 4. select HVG
#sc.pp.highly_variable_genes(adata,min_mean=0.0125,max_mean=3,min_disp=0.5)
sc.pp.highly_variable_genes(adata, n_top_genes = 3000, batch_key="Sample") 
pd.value_counts(adata.var['highly_variable'])
# remove TRXV & IGXV from HVG
adata.var["gene_symbols"] = adata.var.index
adata.var["TRXV"]=adata.var_names.str.startswith('TRAV' or 'TRBV' or 'TRDV' or 'TRGV')
pattern=re.compile(r'IG[A-Z]V')
adata.var["IGXV"]=adata.var["gene_symbols"].apply(lambda x: False if re.search(pattern, x)==None else True)
for i in range(len(list(adata.var["highly_variable"]))):
    if adata.var["TRXV"][i]==True:
        adata.var["highly_variable"][i]=False
    elif adata.var["IGXV"][i]==True:
        adata.var["highly_variable"][i]=False
pd.value_counts(adata.var['highly_variable']) # 2836
adata.raw = adata
adata = adata[:, adata.var['highly_variable']]


# 5. regress out 
sc.pp.regress_out(adata,['total_counts','n_genes','pct_counts_mt','G2M_score','S_score']) #

# 6. scale
sc.pp.scale(adata, max_value=10)


# 7. linar dimensionality reduction
sc.tl.pca(adata, svd_solver='arpack')
print("pca is ok!")
#
df = adata.to_df()
print(df.agg(['min', 'max']))


# 8. save to file
adata.write("/path/UCART_SLE_8Samples_QC_PCA.h5ad")
print(adata)

In [ ]:
# BBKNN
adata = sc.read("/path/UCART_SLE_8Samples_QC_PCA.h5ad")
sce.pp.bbknn(adata, batch_key='Sample', use_rep='X_pca', n_pcs=32, approx=True, pynndescent_n_neighbors=8)
sc.tl.umap(adata,min_dist=0.2,spread=2) #
sc.tl.leiden(adata,resolution=0.5,key_added='leiden_0.5')
adata.obs["leiden_0.5"] = adata.obs["leiden_0.5"].astype(str)

adata.write("/path/UCART_SLE_8Samples_QC_PCA_BBKNN.h5ad")